In [1]:
import json

from torch import Tensor
import torch
from transformers import AutoTokenizer, BloomModel

In [2]:
with open('/home/opc/final_data.json') as f:
    # Load JSON data
    data = json.load(f)
    # Print the loaded data
    #print(data)
review_text_only = [entry['review_text'] for entry in data]
genre = [entry['genre'] for entry in data]
description = [entry['description'] for entry in data]


In [3]:


tokenizer = AutoTokenizer.from_pretrained('izhx/udever-bloom-1b1')
model = BloomModel.from_pretrained('izhx/udever-bloom-1b1')

boq, eoq, bod, eod = '[BOQ]', '[EOQ]', '[BOD]', '[EOD]'
eoq_id, eod_id = tokenizer.convert_tokens_to_ids([eoq, eod])

if tokenizer.padding_side != 'left':
    print('!!!', tokenizer.padding_side)
    tokenizer.padding_side = 'left'


def encode(texts: list, is_query: bool = True, max_length=300):
    bos = boq if is_query else bod
    eos_id = eoq_id if is_query else eod_id
    texts = [bos + t for t in texts]
    encoding = tokenizer(
        texts, truncation=True, max_length=max_length - 1, padding=True
    )
    for ids, mask in zip(encoding['input_ids'], encoding['attention_mask']):
        ids.append(eos_id)
        mask.append(1)
    inputs = tokenizer.pad(encoding, return_tensors='pt')
    with torch.inference_mode():
        outputs = model(**inputs)
        embeds = outputs.last_hidden_state[:, -1]
    return embeds




tokenizer_config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.26G [00:00<?, ?B/s]

In [4]:
review_text_embeds_udever_bloom = encode(review_text_only)
description_embeds_udever_bloom = encode(description)
genre_embeds_udever_bloom = encode(genre)

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [5]:
review_text_embeds_udever_bloom.head(10)

AttributeError: 'Tensor' object has no attribute 'head'